In [1]:
import argparse
import time
from functools import partial

import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
import numpy as np



In [2]:
words = open('names.txt').read().splitlines()
len(words)

1356048

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [4]:


blocksize = 3 # Context length for prediction
X, Y = [], [] # X: Input, Y: Label to the neural net

for w in words:
  # print(w)
  context = [0] * blocksize
  for ch in w + '.':
    ix = stoi[ch] # Prediction
    X.append(context)
    Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]

X = mx.array(X)
Y = mx.array(Y)

# X.shape, X.dtype, Y.shape, Y.dtype

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1058130d0>>
Traceback (most recent call last):
  File "/Volumes/code/envs/mlx/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


((9798219, 3), mlx.core.int32, (9798219,), mlx.core.int32)

In [5]:
X

array([[0, 0, 0],
       [0, 0, 5],
       [0, 5, 13],
       ...,
       [9, 17, 21],
       [17, 21, 1],
       [21, 1, 14]], dtype=int32)

In [17]:
from dataclasses import dataclass

@dataclass
class ModelConfig:
  nInputLayer: int = 1
  nHiddenLayers: list[int] = None
  nOutputLayer: int = 1

In [22]:
from typing import Any


class BigramModel(nn.Module):
  def __init__(self, config: ModelConfig):
    super().__init__()
    self.layers = nn.Linear(config.nInputLayer, config.nOutputLayer, bias=False)
  
  def __call__(self, ix):
    logits = self.layers[ix] * ix
    counts = logits.exp()
    probs = counts / counts.sum(axis=1, keepdims=True)
    loss = - probs[ix].mean().log()
    return probs


In [25]:
config = ModelConfig(nInputLayer=27, nOutputLayer=27)
bigram = BigramModel(config=config)

In [30]:
def loss(x, y):
  y_hat = bigram(x)
  print(type(y_hat))
  return (y_hat - y).square().mean()


In [53]:
params= bigram.parameters()
layers = params["layers"]
layers
weight = layers["weight"]
weight.shape

(27, 27)

In [55]:
lossNgrad = nn.value_and_grad(bigram, loss) 

In [57]:
lossNgrad(words[:2], words[:2])

TypeError: unhashable type: 'list'

In [58]:
print(bigram)

BigramModel(
  (layers): Linear(input_dims=27, output_dims=27, bias=False)
)


How ML frameworksk lower down to hardware?


Pytorch: Eager eval, Tracks the graphs directly through graphs -> supporting multiple backends (metal perf framework)
TF: Not sure
S4TF: Autograd engine is built into the language. Mark functions as differentiable, func.backwards(),  -> LLVM
JAX: Function, mark it as grad, grad(function),  --> XLA
MLX: Similar to numpy,  --> (LLVM*)
XLA